In [3]:
from tqdm.notebook import tqdm
import nltk
import json
# print('NLTK Version: %s' % (nltk.__version__))
nltk.download('stopwords')
nltk_stopwords = nltk.corpus.stopwords.words('english')
nltk_stopwords += ["like", "gone", "did", "going", "would", "could", "get", "in", "up", "may", "wanter"]
#print(nltk_stopwords)
# print(nltk_stopwords)
import configparser
import sys

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data = []
concept_vocab = set()
config = configparser.ConfigParser()
config.read("paths.cfg")
with open(config["paths"]["concept_vocab"], "r", encoding="utf8") as f:
    cpnet_vocab = set([l.strip() for l in list(f.readlines())])

In [5]:
#cpnet_vocab
path = './../datasets/arc_data/'

In [6]:
with open(path+"arc_concept_final.mcp",'r') as file:
    concepts = json.load(file)

In [11]:
def prunning(data):
    prune_data = []
    for item in data:
        qc = item["premise"]
        prune_qc = []
        for c in qc:
            if c[-2:] == "er" and c[:-2] in qc:
                continue
            if c[-1:] == "e" and c[:-1] in qc:
                continue
            have_stop = False
            for t in c.split("_"):
                if t in nltk_stopwords:
                    have_stop = True
            if not have_stop and c in cpnet_vocab:
                prune_qc.append(c)

        ac = item["hypothesis"]
        prune_ac = []
        for c in ac:
            if c[-2:] == "er" and c[:-2] in ac:
                continue
            if c[-1:] == "e" and c[:-1] in ac:
                continue
            all_stop = True
            for t in c.split("_"):
                if t not in nltk_stopwords:
                    all_stop = False
            if not all_stop and c in cpnet_vocab:
                prune_ac.append(c)

        item["premise"] = prune_qc
        item["hypothesis"] = prune_ac

        prune_data.append(item)
    return prune_data

In [12]:
def process_prune(examples):
    final_res=[]
    for example in tqdm(examples):
        con = []
        for i in example:
            x = prunning(i)
            con.append(x)
        final_res.append(con)
    return final_res

In [13]:
final_con = process_prune(concepts)

In [15]:
len(final_con[0])

4

In [16]:
len(final_con)

3353

In [17]:
import jsbeautifier
opts = jsbeautifier.default_options()
opts.indent_size = 2

with open(path+'arc_train_final.mcp', 'w') as fp:
    # json.dump(prune_data, f)
    fp.write(jsbeautifier.beautify(json.dumps(final_con), opts))